In [3]:
# !pip install selenium
# !pip install webdriver-manager
# !pip install bs4
# !pip install xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from lime.lime_tabular import LimeTabularExplainer
import requests
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
data_true_location_eliel = '~/Downloads/DHDS/TP Final/git_fake_news/data/True.csv'
data_fake_location_eliel = '~/Downloads/DHDS/TP Final/git_fake_news/data/Fake.csv'
# data_true_location_lucas = 'C:\\Users\\Lucas Choconi\\Documents\\DH\\Datasets\\fakenews\True.csv'
# data_fake_location_lucas = 'C:\\Users\\Lucas Choconi\\Documents\\DH\\Datasets\\fakenews\Fake.csv'
# data_true_location_ger = ''
# data_fake_location_ger =  ''

In [ ]:
file_names_true = pd.read_csv(data_true_location_eliel)
file_names_true['real'] = 'True'
file_names_fake = pd.read_csv(data_fake_location_eliel)
file_names_fake['real'] = 'Fake'
data = file_names_fake.append(file_names_true, ignore_index=True)
data.head(2)

In [ ]:
data.isna().sum()

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.2)
sample_size = 5000
data_train = data_train.sample(sample_size)
data_test = data_test.sample(sample_size)

In [ ]:
stopwords_en = stopwords.words('english')

In [ ]:
def manual_stopword_filtering(stopwords_en): 
    stopwords_en.append('reuter')
    stopwords_en.append('said')
    stopwords_en.append('Reuters')
    stopwords_en.append('via')
    stopwords_en.append('imag')
    stopwords_en.append('https')
    stopwords_en.append('com')
    stopwords_en.append('one')
    stopwords_en.append('u')
    stopwords_en.append('also')
    stopwords_en.append('would')
    stopwords_en.append('featur')
    stopwords_en.append('pic')
    stopwords_en.append('us')
    stopwords_en.append('wednesday')
    stopwords_en.append('friday')
    stopwords_en.append('monday')
    stopwords_en.append('tuesday')
    stopwords_en.append('saturday')
    stopwords_en.append('sunday')
    stopwords_en.append('thursday')
    stopwords_en.append('getti')
    stopwords_en.append('read')
    stopwords_en.append('gop')
    stopwords_en.append('watch')
    stopwords_en.append('donald')
    stopwords_en.append('trump')
    stopwords_en.append('hillari')
    stopwords_en.append('mr')
    stopwords_en.append('accord')
    stopwords_en.append('america')
    stopwords_en.append('seem')
    stopwords_en.append('youtub')
    stopwords_en.append('21st')
    return stopwords_en

In [ ]:
stopwords_en_filtered = manual_stopword_filtering(stopwords_en)

In [ ]:
porter = PorterStemmer()
lancaster= LancasterStemmer()

stopwords_en_porter = [porter.stem(x) for x in stopwords_en]
stopwords_en_lancaster = [lancaster.stem(x) for x in stopwords_en]
stopwords_en_filtered_poter = [porter.stem(x) for x in stopwords_en_filtered]
stopwords_en_filtered_lancaster = [lancaster.stem(x) for x in stopwords_en_filtered]

In [ ]:
def porter_tokenizer(text):
    words = word_tokenize(text) # tokenize
    porter = PorterStemmer()
    stemmed_words=[porter.stem(word) for word in words]  # obtain stem words
    return stemmed_words

In [ ]:
vectorizer=CountVectorizer(stop_words = stopwords_en, lowecase = true, strip_accents='unicode', tokenizer = porter_tokenizer);
vectorizer.fit(data_train['text'])
vectorizer_matrix = vectorizer.transform(data_train['text'])
pd.DataFrame(vectorizer_matrix.todense(), columns=vectorizer.get_feature_names()).head(2)

In [ ]:
stopwords_en = stopwords.words('english')

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
mask_pos = np.array(Image.open(path.join(d, "Thumbs.png")))
mask_neg = np.array(Image.open(path.join(d, "Thumbsdw.png")))

text_fake = " ".join(text for text in file_names[file_names['real']=='Fake'].text)
text_true = " ".join(text for text in file_names[file_names['real']=='True'].text)

wordcloud_fake = WordCloud(stopwords=stopwords_en,  mask=mask_neg, max_words=500, background_color="white").generate(text_fake)
wordcloud_true = WordCloud(stopwords=stopwords_en, mask=mask_pos, max_words=500, background_color="white").generate(text_true)

In [ ]:
def blue_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    blues = "hsl(215,100%%, %d%%)" % np.random.choice(np.arange(25, 65, 5, dtype=int))
    reds = "hsl(0,100%%, %d%%)" % np.random.choice(np.arange(40, 80, 5, dtype=int))
    choice = np.random.choice([0, 1])
    if choice == 0:
        return blues
    else:
        return reds

fig, axs = plt.subplots(1, 2,figsize=(20,10))  

axs[1].imshow(wordcloud_fake.recolor(color_func=blue_color_func),interpolation="bilinear")
axs[1].axis("off")
axs[1].title.set_text('FAKE')
axs[0].imshow(wordcloud_true.recolor(color_func=blue_color_func),interpolation="bilinear")
axs[0].axis("off")
axs[0].title.set_text('TRUE')
plt.show()

In [ ]:
def clean_review(review_text, tokenizer, stemmer, stopwords):    
    
    #tokens (eliminamos todos los signos de puntuación)
    words = tokenizer.tokenize(review_text)
    
    # stemming: raiz y minúsculas:
    stem_words = [stemmer.stem(x) for x in words]
    
    # eliminamos stopwords (ya pasaron por stem)
    clean_words = [x for x in stem_words if x not in stopwords]
    
    result = " ".join(clean_words)
    
    return(result)

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
englishStemmer = SnowballStemmer("english")
stopwords_en = stopwords.words('english')
stopwords_en.append('reuter')
stopwords_en.append('said')
stopwords_en.append('Reuters')
stopwords_en.append('via')
stopwords_en.append('imag')
stopwords_en.append('https')
stopwords_en.append('com')
stopwords_en.append('one')
stopwords_en.append('u')
stopwords_en.append('also')
stopwords_en.append('would')
stopwords_en.append('featur')
stopwords_en.append('pic')
stopwords_en.append('us')
stopwords_en.append('wednesday')
stopwords_en.append('friday')
stopwords_en.append('monday')
stopwords_en.append('tuesday')
stopwords_en.append('saturday')
stopwords_en.append('sunday')
stopwords_en.append('thursday')
stopwords_en.append('getti')
stopwords_en.append('read')
stopwords_en.append('gop')
stopwords_en.append('watch')
stopwords_en.append('donald')
stopwords_en.append('trump')
stopwords_en.append('hillari')
stopwords_en.append('mr')
stopwords_en.append('accord')
stopwords_en.append('america')
stopwords_en.append('seem')
stopwords_en.append('youtub')
stopwords_en.append('21st')
stopwords_en_stem = [englishStemmer.stem(x) for x in stopwords_en]

In [ ]:
tqdm.pandas()

clean_train = data_train.text.progress_apply(lambda x: clean_review(x, tokenizer, englishStemmer, stopwords_en_stem))
clean_test = data_test.text.progress_apply(lambda x: clean_review(x, tokenizer, englishStemmer, stopwords_en_stem))

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(clean_train)
X_train_sparse = count_vectorizer.transform(clean_train)
X_test_sparse = count_vectorizer.transform(clean_test)

In [ ]:
X_train = pd.DataFrame(X_train_sparse.todense(), 
             columns = count_vectorizer.get_feature_names()) 
y_train = data_train.real

In [ ]:
X_test = pd.DataFrame(X_test_sparse.todense(), 
             columns = count_vectorizer.get_feature_names()) 
y_test = data_test.real

#### Modelo de Reg. Logistica

In [ ]:
reg_model = LogisticRegression(C = 0.05, solver="newton-cg", penalty="l2")
reg_model.fit(X_train, y_train)
reg_model.score(X_test, y_test)

In [ ]:
myColors = ((0.90, 0.96, 1, 1), (0.70, 0.87, 1, 1), (0, 0.40, 0.75, 0.88))
cmap = LinearSegmentedColormap.from_list('Custom', myColors,10)
cm = confusion_matrix(y_test, reg_model.predict(X_test))
axis_labels = np.sort(y_test.unique())
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

In [ ]:
Coef_reg_model = pd.DataFrame(np.transpose(reg_model.coef_), index=X_train.columns, columns=['Coef'])
Coef_reg_model['coef_abs'] = abs(Coef_reg_model['Coef'])
Coef_reg_model = Coef_reg_model.reset_index(-1)
Coef_reg_model['words'] = Coef_reg_model['index'] 
Coef_reg_model['freq'] = (round(Coef_reg_model['coef_abs'] / Coef_reg_model.coef_abs.sum() * Coef_reg_model.shape[0] * 1000 , 0)).apply(lambda x: int(x))
Coef_reg_model['odds_direction'] = Coef_reg_model['Coef'].apply(lambda x: 'Negative' if x<0 else 'Positive')
Coef_reg_model_positive = Coef_reg_model[Coef_reg_model['odds_direction']=='Positive']
Coef_reg_model_positive = Coef_reg_model_positive.drop(['Coef' ,'coef_abs', 'index', 'odds_direction'], axis=1)
Coef_reg_model_negative = Coef_reg_model[Coef_reg_model['odds_direction']=='Negative']
Coef_reg_model_negative = Coef_reg_model_negative.drop(['Coef' ,'coef_abs', 'index', 'odds_direction'], axis=1)

In [ ]:
tuples_pos = Coef_reg_model_positive.set_index('words').T.to_dict('index')
tuples_neg = Coef_reg_model_negative.set_index('words').T.to_dict('index')

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
mask_pos = np.array(Image.open(path.join(d, "Thumbs.png")))
mask_neg = np.array(Image.open(path.join(d, "Thumbsdw.png")))

wordcloud_positive = WordCloud(max_words=500, mask=mask_pos, background_color="white").generate_from_frequencies(dict(tuples_pos['freq']))
wordcloud_negative = WordCloud(max_words=500, mask=mask_neg,background_color="white").generate_from_frequencies(dict(tuples_neg['freq']))

fig, axs = plt.subplots(1, 2,figsize=(20,10))
# Display the generated image:
axs[0].imshow(wordcloud_positive.recolor(color_func=blue_color_func, random_state=3),interpolation="bilinear")
axs[0].axis("off")
axs[0].title.set_text('Positive')
axs[1].imshow(wordcloud_negative.recolor(color_func=blue_color_func, random_state=3),interpolation="bilinear")
axs[1].axis("off")
axs[1].title.set_text('Negative')
plt.show()

## Naive Bayes

In [ ]:
NB_model = MultinomialNB()
NB_model.fit(X_train, y_train)
NB_model.score(X_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, NB_model.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

## Modelo de Reg. Logistica Truncando palabras

In [ ]:
svd = TruncatedSVD(n_components = 200);

X_train_svd = svd.fit_transform(X_train)
X_test_svd = svd.transform(X_test)

In [ ]:
display(svd.explained_variance_ratio_.sum())
svd.explained_variance_ratio_

In [ ]:
reg_model_svd = LogisticRegression(C = 0.05, solver="newton-cg", penalty="l2")
reg_model_svd.fit(X_train_svd, y_train)
reg_model_svd.score(X_test_svd, y_test)

In [ ]:
cm = confusion_matrix(y_test, reg_model_svd.predict(X_test_svd))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

## Modelo de Reg. Logistica con conjunto de palabras

In [ ]:
count_vectorizer_bigram = CountVectorizer(ngram_range = (1, 2))
count_vectorizer_bigram.fit(clean_train)
X_train_bigram_sparse = count_vectorizer_bigram.transform(clean_train)
X_test_bigram_sparse = count_vectorizer_bigram.transform(clean_test)

In [ ]:
reg_model_bigram = LogisticRegression(C = 0.05, solver="newton-cg", penalty="l2")
reg_model_bigram.fit(X_train_bigram_sparse, y_train)
reg_model_bigram.score(X_test_bigram_sparse, y_test)

In [ ]:
cm = confusion_matrix(y_test, reg_model_bigram.predict(X_test_bigram_sparse))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

## Decision Tree Classifier

In [ ]:
my_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
my_tree.fit(X_train, y_train)
my_tree.score(X_test, y_test)

In [ ]:
cm = confusion_matrix(y_test, my_tree.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');

In [ ]:
plt.figure(figsize=(20,6))
tree.plot_tree(my_tree,feature_names = X_train.columns,filled=True,rounded=True, fontsize = 10)
plt.show()

## Random Decision Tree Classifier and Extra random

In [ ]:
'''random_forest = RandomForestClassifier(n_estimators=100, 
                                      criterion='gini', 
                                      max_depth = 4, 
                                      bootstrap=True, 
                                      n_jobs = -1, 
                                      random_state = 127,
                                      max_samples= 0.3)
random_forest.fit(X_train, y_train)
random_forest.score(X_test, y_test)'''

In [ ]:
'''cm = confusion_matrix(y_test, random_forest.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');'''

In [ ]:
'''extra_random_forest = ExtraTreesClassifier(n_estimators=100, 
                                      criterion='gini', 
                                      max_depth = 4, 
                                      bootstrap=True, 
                                      n_jobs = -1, 
                                      random_state = 127,
                                      max_samples= 0.3)
extra_random_forest.fit(X_train, y_train)
extra_random_forest.score(X_test, y_test)'''

In [ ]:
'''cm = confusion_matrix(y_test, extra_random_forest.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');'''

## Boosting de Modelos

In [ ]:
'''base_classifier = ExtraTreesClassifier(n_estimators=100, 
                                      criterion='gini', 
                                      max_depth = 4, 
                                      bootstrap=True, 
                                      n_jobs = -1, 
                                      random_state = 127,
                                      max_samples= 0.3)

boost_tree = AdaBoostClassifier(base_estimator = base_classifier, 
                            n_estimators = 200,
                            learning_rate = 0.8,                                       
                            random_state = 127)
boost_tree.fit(X_train, y_train) 
boost_tree.score(X_test, y_test)'''

In [ ]:
'''cm = confusion_matrix(y_test, boost_tree.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');'''

In [ ]:
'''gb_classifier = GradientBoostingClassifier(loss = 'deviance',
                                           learning_rate=0.6,
                                           n_estimators = 200,
                                           subsample=1,
                                           criterion='mse',
                                           random_state = 127)

gb_classifier.fit(X_train, y_train)
gb_classifier.score(X_test, y_test)'''

In [ ]:
'''cm = confusion_matrix(y_test, gb_classifier.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');'''

In [ ]:
'''y_train_xg = y_train.apply(lambda x: 1 if x=='Fake' else 0)
y_test_xg = y_test.apply(lambda x: 1 if x=='Fake' else 0)
model_xg = XGBClassifier(n_jobs=-1, use_label_encoder=False, n_estimators= 100, max_depth=4, learning_rate= 0.05)
model_xg.fit(X_train,y_train_xg)
model_xg.score(X_test, y_test_xg)'''

In [ ]:
'''cm = confusion_matrix(y_test_xg, model_xg.predict(X_test))
sns.heatmap(cm, xticklabels=axis_labels, yticklabels=axis_labels, cbar=False, annot=True, cmap=cmap, fmt='g')
plt.ylabel('Etiquetas reales')
plt.xlabel('Etiquetas predichas');'''

## A traves de webscraping se buscan noticias para obtener más datos de testing

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)

def get_text_from_url_bbc(url):
    wd.get(url)

    time.sleep(1)

    soup = BeautifulSoup(wd.page_source)
    text = soup.find_all('div', {'data-component': 'text-block'}) #(?P<text>\>(.*?)\<)
    text = reversed(text) 
    text2 = ""
    for i in text:
        try:
            text2 = i.text+text2
        except:
            text2 = ""+text2
    return text2    

In [ ]:

warnings.filterwarnings("ignore")
#https://newsapi.org/docs
response = requests.get("https://newsapi.org/v2/top-headlines?sources=bbc-news&apiKey=1f83e742c8804de0a5d427510829f79b")

list_urls = []
for i in range(0,len(response.json()['articles'])):
    list_urls.append(response.json()['articles'][i]['url'])  
    
list_texts = []
for i in range(0,len(list_urls)):
    list_texts.append(get_text_from_url_bbc(list_urls[i]))

In [ ]:
data_texts_text = pd.DataFrame(list_texts, columns=['text'])
data_texts_text['len'] = data_texts_text.text.apply(lambda x: len(x))
data_texts_text['real'] = 'True'
data_texts_text = data_texts_text[data_texts_text.len>0]
clean_texts_test = data_texts_text.text.progress_apply(lambda x: clean_review(x, tokenizer, englishStemmer, stopwords_en_stem))

X_test_sparse_texts = count_vectorizer.transform(clean_texts_test)
X_test_texts = pd.DataFrame(X_test_sparse_texts.todense(), 
             columns = count_vectorizer.get_feature_names()) 
X_test_svd_texts = svd.transform(X_test_texts)
X_test_bigram_sparse_texts = count_vectorizer_bigram.transform(clean_texts_test)

In [ ]:
data_texts_text['predict_reg_base'] = reg_model.predict(X_test_texts)
data_texts_text['predict_prob_reg_base'] = reg_model.predict_proba(X_test_texts).T[0]

data_texts_text['predict_svd'] = reg_model_svd.predict(X_test_svd_texts)
data_texts_text['predict_prob_svd'] = reg_model_svd.predict_proba(X_test_svd_texts).T[0]

data_texts_text['predict_bigram'] = reg_model_bigram.predict(X_test_bigram_sparse_texts)
data_texts_text['predict_prob_bigram'] = reg_model_bigram.predict_proba(X_test_bigram_sparse_texts).T[0]

data_texts_text['predict_my_tree'] = my_tree.predict(X_test_texts)
data_texts_text['predict_prob_my_tree'] = my_tree.predict_proba(X_test_texts).T[0]

data_texts_text

In [ ]:
X_train_explainer = np.array(X_train)
explainer = LimeTabularExplainer(X_train_explainer, 
                                 mode = "classification",
                                 training_labels = y_train,
                                 feature_names = X_train.columns,
                                 discretize_continuous=False)

#i = 13
#data_row = np.array(X_test.iloc[i])
data_row = np.array(X_test_texts.iloc[4])
explanation = explainer.explain_instance(data_row, reg_model.predict_proba, num_features=10)
explanation.as_pyplot_figure();

In [ ]:
data_texts_text.text.iloc[4]